In [ ]:
import numpy as np
x_npy_path = "human/preprocess/X_processed_minmax_quantile.npy"
y_npy_path = "human/preprocess/y.npy"

y = np.load(y_npy_path)

X_full = np.load(x_npy_path); y = np.load(y_npy_path)
print(X_full.shape)
print(y.shape)
D = X_full.shape[1]

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
seed = np.random.randint(0, 2**32 - 1)
n_jobs = -1
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_full, y), start=1):
    X_tr, X_te = X_full[tr_idx], X_full[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy: {acc:.4f}")

print(f"Mean accuracy (2-fold): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")


(632, 2000)
(632,)
Fold 1 accuracy: 0.9177
Fold 2 accuracy: 0.9177
Mean accuracy (2-fold): 0.9177 ± 0.0000


# LOCO

In [ ]:
from Flow_Disentangle_Feature_Importance import LOCOEstimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np

estimator1 = LOCOEstimator_cls(
        random_state=seed,
        regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                )

)

phi_0_loco, se_0_loco = estimator1.importance(X_full, y)


print("Feature\tLOCO φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_0_loco, se_0_loco)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of LOCO: {D* np.mean(phi_0_loco)}")



phi_0_loco_test = phi_0_loco 

se_0_loco_test = se_0_loco 

z_score_0_loco = phi_0_loco_test / se_0_loco_test

p_value_0_loco = 1 - norm.cdf(z_score_0_loco)
rounded_p_value_0_loco = np.round(p_value_0_loco, 3)

print(rounded_p_value_0_loco)


alpha = 0.05 / 2000  


mask = (p_value_0_loco <= alpha).astype(int)

print(mask)

/root/miniconda3/lib/python3.12/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Feature	LOCO φ	StdError
  0	 0.0000	 0.0000
  1	 0.0000	 0.0000
  2	 0.0000	 0.0000
  3	 0.0000	 0.0022
  4	 0.0000	 0.0000
  5	 0.0000	 0.0000
  6	 0.0000	 0.0000
  7	 0.0000	 0.0000
  8	 0.0000	 0.0000
  9	 0.0000	 0.0000
 10	 0.0000	 0.0000
 11	 0.0000	 0.0000
 12	 0.0000	 0.0000
 13	 0.0000	 0.0000
 14	 0.0000	 0.0000
 15	 0.0000	 0.0000
 16	 0.0000	 0.0000
 17	 0.0000	 0.0000
 18	 0.0000	 0.0000
 19	 0.0000	 0.0000
 20	 0.0000	 0.0000
 21	 0.0000	 0.0000
 22	 0.0000	 0.0000
 23	 0.0000	 0.0000
 24	 0.0000	 0.0000
 25	 0.0000	 0.0000
 26	 0.0000	 0.0000
 27	 0.0000	 0.0000
 28	 0.0000	 0.0000
 29	 0.0000	 0.0000
 30	 0.0000	 0.0000
 31	 0.0000	 0.0000
 32	 0.0000	 0.0000
 33	 0.0000	 0.0000
 34	 0.0000	 0.0000
 35	 0.0000	 0.0000
 36	 0.0000	 0.0000
 37	 0.0000	 0.0000
 38	 0.0000	 0.0000
 39	 0.0000	 0.0000
 40	 0.0000	 0.0000
 41	 0.0000	 0.0000
 42	 0.0000	 0.0000
 43	 0.0000	 0.0000
 44	 0.0000	 0.0000
 45	 0.0000	 0.0000
 46	 0.0000	 0.0000
 47	 0.0000	 0.0000
 48	 0.0000	 0.0

# CPI

In [ ]:
from Flow_Disentangle_Feature_Importance import CPIEstimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np

estimator2 = CPIEstimator_cls(
        random_state=seed,
        regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                )
)

phi_0_cpi, se_0_cpi = estimator2.importance(X_full, y)

print("Feature\tCPI φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_0_cpi, se_0_cpi)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of CPI: {D* np.mean(phi_0_cpi)}")

phi_0_cpi_test = phi_0_cpi 
se_0_cpi_test = se_0_cpi 

z_score_0_cpi = phi_0_cpi_test / se_0_cpi_test

p_value_0_cpi = 1 - norm.cdf(z_score_0_cpi)
rounded_p_value_0_cpi = np.round(p_value_0_cpi, 3)

print(rounded_p_value_0_cpi)

alpha = 0.05 / 2000  

mask = (p_value_0_cpi <= alpha).astype(int)

print(mask)


CPI conditional permutation: 100%|██████████| 2000/2000 [1:26:38<00:00,  2.60s/it]

Feature	CPI φ	StdError
  0	 0.0000	 0.0000
  1	 0.0001	 0.0001
  2	 0.0000	 0.0000
  3	 0.0000	 0.0000
  4	 0.0000	 0.0000
  5	 0.0000	 0.0000
  6	 0.0000	 0.0000
  7	 0.0000	 0.0000
  8	 0.0000	 0.0000
  9	 0.0000	 0.0000
 10	 0.0000	 0.0000
 11	 0.0000	 0.0000
 12	 0.0000	 0.0000
 13	 0.0000	 0.0000
 14	 0.0000	 0.0000
 15	 0.0000	 0.0000
 16	 0.0000	 0.0000
 17	 0.0000	 0.0000
 18	 0.0000	 0.0000
 19	 0.0000	 0.0000
 20	 0.0000	 0.0000
 21	 0.0001	 0.0002
 22	 0.0000	 0.0000
 23	 0.0000	 0.0000
 24	 0.0000	 0.0000
 25	 0.0000	 0.0000
 26	 0.0000	 0.0000
 27	 0.0000	 0.0000
 28	 0.0000	 0.0000
 29	 0.0000	 0.0000
 30	 0.0000	 0.0000
 31	 0.0000	 0.0000
 32	 0.0000	 0.0000
 33	 0.0000	 0.0000
 34	 0.0000	 0.0000
 35	 0.0000	 0.0000
 36	 0.0001	 0.0001
 37	 0.0000	 0.0000
 38	 0.0000	 0.0000
 39	 0.0001	 0.0000
 40	 0.0000	 0.0000
 41	 0.0000	 0.0000
 42	 0.0000	 0.0000
 43	 0.0000	 0.0000
 44	 0.0000	 0.0000
 45	 0.0000	 0.0000
 46	 0.0000	 0.0000
 47	 0.0000	 0.0000
 48	 0.0000	 0.00

# FDFI

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from Flow_Disentangle_Feature_Importance import FlowMatchingModel

model = FlowMatchingModel(
    X=X_full,
    dim=D,
    device=device,
    hidden_dim=512,        
    time_embed_dim=64,     
    num_blocks=1,
    use_bn=False,
    seed=seed
)
model.fit(num_steps=10000, batch_size=256, lr=1e-3, show_plot=False)

from Flow_Disentangle_Feature_Importance import CPI_Flow_Model_Estimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np

estimator3 = CPI_Flow_Model_Estimator_cls(
    random_state=seed,
    regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                ),
    flow_model=model
)

phi_x_cpi, se_x_cpi = estimator3.importance(X_full, y)

print("Feature\tFDFI φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_x_cpi, se_x_cpi)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of FDFI: {D* np.mean(phi_x_cpi)}")

phi_x_cpi_test = phi_x_cpi 

se_x_cpi_test = se_x_cpi 

z_score_x_cpi = phi_x_cpi_test / se_x_cpi_test

p_value_x_cpi = 1 - norm.cdf(z_score_x_cpi)
rounded_p_value_x_cpi = np.round(p_value_x_cpi, 3)

print(rounded_p_value_x_cpi)

alpha = 0.05 / 2000  

mask = (p_value_x_cpi <= alpha).astype(int)

print(mask)



CPI@Z (resample) → decode → X (cls CE): 100%|██████████| 2000/2000 [25:40<00:00,  1.30it/s]
/root/autodl-tmp/Module_Flow_Disentangle_Feature_Importance/Flow_Disentangle_Feature_Importance/Flow_Matching/flow_matching.py:213: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  t_tensor = torch.tensor([[float(t_scalar)]], dtype=torch.float32, device=self.device)
CPI@Z (resample) → decode → X (cls CE): 100%|██████████| 2000/2000 [23:42<00:00,  1.41it/s]


Feature	FDFI φ	StdError
  0	 0.0004	 0.0003
  1	 0.0007	 0.0003
  2	 0.0004	 0.0003
  3	 0.0008	 0.0003
  4	 0.0006	 0.0003
  5	 0.0005	 0.0003
  6	 0.0006	 0.0004
  7	 0.0004	 0.0003
  8	 0.0004	 0.0003
  9	 0.0004	 0.0002
 10	 0.0005	 0.0003
 11	 0.0009	 0.0004
 12	 0.0004	 0.0003
 13	 0.0003	 0.0002
 14	 0.0006	 0.0003
 15	 0.0004	 0.0002
 16	 0.0008	 0.0003
 17	 0.0011	 0.0004
 18	 0.0010	 0.0004
 19	 0.0003	 0.0002
 20	 0.0005	 0.0002
 21	 0.0015	 0.0006
 22	 0.0011	 0.0004
 23	 0.0006	 0.0003
 24	 0.0011	 0.0004
 25	 0.0010	 0.0003
 26	 0.0009	 0.0003
 27	 0.0007	 0.0003
 28	 0.0009	 0.0004
 29	 0.0007	 0.0003
 30	 0.0005	 0.0002
 31	 0.0004	 0.0002
 32	 0.0004	 0.0002
 33	 0.0009	 0.0004
 34	 0.0006	 0.0003
 35	 0.0008	 0.0004
 36	 0.0009	 0.0004
 37	 0.0004	 0.0003
 38	 0.0004	 0.0003
 39	 0.0009	 0.0003
 40	 0.0005	 0.0003
 41	 0.0005	 0.0003
 42	 0.0007	 0.0003
 43	 0.0006	 0.0003
 44	 0.0004	 0.0003
 45	 0.0009	 0.0004
 46	 0.0003	 0.0002
 47	 0.0005	 0.0003
 48	 0.0003	 0.0

# DFI

In [ ]:
from Flow_Disentangle_Feature_Importance import  DFIEstimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np
print(X_full.shape)
estimator4 = DFIEstimator_cls(
    random_state=seed,
    regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                )
)

phi_x_dfi, se_x_dfi = estimator4.importance(X_full, y)


print("Feature\tDFI φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_x_dfi, se_x_dfi)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of DFI: {D* np.mean(phi_x_dfi)}")

phi_x_dfi_test = phi_x_dfi 

se_x_dfi_test = se_x_dfi 

z_score_x_dfi = phi_x_dfi_test / se_x_dfi_test


p_value_x_dfi = 1 - norm.cdf(z_score_x_dfi)
rounded_p_value_x_dfi = np.round(p_value_x_dfi, 3)

print(rounded_p_value_x_dfi)

alpha = 0.05 / 2000  

mask = (p_value_x_dfi <= alpha).astype(int)

print(mask)



(632, 2000)
Feature	DFI φ	StdError
  0	 0.0198	 0.0006
  1	 0.0160	 0.0005
  2	 0.0099	 0.0003
  3	 0.0124	 0.0003
  4	 0.0108	 0.0003
  5	 0.0094	 0.0003
  6	 0.0129	 0.0004
  7	 0.0091	 0.0003
  8	 0.0158	 0.0005
  9	 0.0051	 0.0002
 10	 0.0143	 0.0005
 11	 0.0158	 0.0005
 12	 0.0119	 0.0004
 13	 0.0048	 0.0001
 14	 0.0117	 0.0004
 15	 0.0054	 0.0002
 16	 0.0069	 0.0002
 17	 0.0069	 0.0002
 18	 0.0069	 0.0002
 19	 0.0047	 0.0001
 20	 0.0081	 0.0002
 21	 0.0143	 0.0004
 22	 0.0078	 0.0003
 23	 0.0078	 0.0003
 24	 0.0078	 0.0003
 25	 0.0078	 0.0003
 26	 0.0078	 0.0003
 27	 0.0078	 0.0003
 28	 0.0078	 0.0003
 29	 0.0111	 0.0003
 30	 0.0042	 0.0001
 31	 0.0054	 0.0002
 32	 0.0094	 0.0003
 33	 0.0109	 0.0003
 34	 0.0087	 0.0003
 35	 0.0046	 0.0001
 36	 0.0170	 0.0005
 37	 0.0060	 0.0002
 38	 0.0162	 0.0006
 39	 0.0041	 0.0001
 40	 0.0119	 0.0004
 41	 0.0099	 0.0004
 42	 0.0082	 0.0003
 43	 0.0152	 0.0005
 44	 0.0056	 0.0002
 45	 0.0162	 0.0006
 46	 0.0143	 0.0005
 47	 0.0115	 0.0004
 48	 

# ad hoc method

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

print("X shape:", X_full.shape)
print("y shape:", y.shape)

n, D = X_full.shape

ctg_names = [
    "LB","AC","FM","UC","DL","DS","DP",
    "ASTV","MSTV","ALTV","MLTV",
    "Width","Min","Max","Nmax","Nzeros","Mode","Mean","Median","Variance","Tendency"
]
feature_names = ctg_names if D == 21 else [f"f{i}" for i in range(D)]
X_df = pd.DataFrame(X_full, columns=feature_names)

corr = spearmanr(X_df).correlation
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1.0)
corr = np.nan_to_num(corr, nan=0.0, posinf=0.0, neginf=0.0)

distance_matrix = 1.0 - np.abs(corr)
linkage = hierarchy.ward(squareform(distance_matrix))

t = 1.201  # 1.075
cluster_ids = hierarchy.fcluster(linkage, t, criterion="distance")
cluster_to_ids = defaultdict(list)
for idx, cid in enumerate(cluster_ids):
    cluster_to_ids[cid].append(idx)

def select_representatives(
    X: np.ndarray,
    y: np.ndarray,
    cluster_to_ids: dict,
    corr_abs: np.ndarray,           # |spearman|
    dist: np.ndarray,               # 1 - |spearman|
    strategy: str = "medoid",
    random_state: int = 0
):
    rng = np.random.RandomState(random_state)
    selected = []

    for cid, ids in cluster_to_ids.items():
        ids = list(ids)
        if len(ids) == 1:
            selected.append(ids[0])
            continue

        if strategy == "first":
            chosen = ids[0]

        elif strategy == "medoid":
            subD = dist[np.ix_(ids, ids)]
            sums = np.sum(subD, axis=1)
            chosen = ids[int(np.argmin(sums))]
        selected.append(chosen)

    return selected

strategy = "medoid"   
selected_feature_idx = select_representatives(
    X_full, y, cluster_to_ids,
    corr_abs=np.abs(corr),
    dist=distance_matrix,
    strategy=strategy,
    random_state=0
)
selected_feature_names = [feature_names[i] for i in selected_feature_idx]
print(f"[{strategy}] selected numbers: {len(selected_feature_idx)}")
print(" selected names:", selected_feature_names)

X_subset = X_full[:, selected_feature_idx]

n_jobs = -1
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")


X shape: (632, 2000)
y shape: (632,)
[medoid] selected numbers: 100
 selected names: ['f29', 'f1', 'f5', 'f156', 'f4', 'f140', 'f1393', 'f8', 'f30', 'f159', 'f11', 'f12', 'f33', 'f28', 'f360', 'f457', 'f590', 'f38', 'f39', 'f631', 'f44', 'f125', 'f49', 'f51', 'f515', 'f162', 'f58', 'f1757', 'f64', 'f304', 'f866', 'f74', 'f1204', 'f77', 'f505', 'f1159', 'f389', 'f99', 'f587', 'f1842', 'f235', 'f151', 'f1968', 'f1144', 'f1072', 'f831', 'f620', 'f160', 'f393', 'f314', 'f856', 'f203', 'f478', 'f220', 'f226', 'f355', 'f250', 'f487', 'f392', 'f447', 'f903', 'f1389', 'f298', 'f702', 'f555', 'f808', 'f1535', 'f684', 'f1009', 'f1932', 'f441', 'f1548', 'f474', 'f534', 'f495', 'f1095', 'f722', 'f838', 'f1319', 'f840', 'f1122', 'f618', 'f786', 'f600', 'f1424', 'f1545', 'f664', 'f670', 'f800', 'f1719', 'f1823', 'f836', 'f1439', 'f1201', 'f1813', 'f1492', 'f925', 'f1587', 'f1940', 'f1585']
Fold 1 accuracy (selected features): 0.9209
Fold 2 accuracy (selected features): 0.9241
Mean accuracy (2-fold, 

In [ ]:
from Flow_Disentangle_Feature_Importance import CPIEstimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np

estimator5 = CPIEstimator_cls(
        random_state=seed,
        regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                )

)

phi_0_cpi_ad_hoc, se_0_cpi_ad_hoc = estimator5.importance(X_subset, y)


print("Feature\tCPI__ad_hoc φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_0_cpi_ad_hoc, se_0_cpi_ad_hoc)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of CPI__ad_hoc: {D* np.mean(phi_0_cpi_ad_hoc)}")


phi_0_cpi_test_ad_hoc = phi_0_cpi_ad_hoc 

se_0_cpi_test_ad_hoc = se_0_cpi_ad_hoc 

z_score_0_cpi_ad_hoc = phi_0_cpi_test_ad_hoc / se_0_cpi_ad_hoc

p_value_0_cpi_ad_hoc = 1 - norm.cdf(z_score_0_cpi_ad_hoc)
rounded_p_value_0_cpi_ad_hoc = np.round(p_value_0_cpi_ad_hoc, 3)

print(rounded_p_value_0_cpi_ad_hoc)

alpha = 0.05 / 100  

mask = (p_value_0_cpi_ad_hoc <= alpha).astype(int)

print(mask)




/root/miniconda3/lib/python3.12/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
CPI conditional permutation: 100%|██████████| 100/100 [00:53<00:00,  1.86it/s]

Feature	CPI__ad_hoc φ	StdError
  0	 0.0000	 0.0000
  1	 0.0010	 0.0010
  2	 0.0000	 0.0000
  3	 0.0000	 0.0001
  4	 0.0000	 0.0001
  5	 0.0000	 0.0000
  6	 0.0000	 0.0000
  7	 0.0000	 0.0000
  8	 0.0000	 0.0000
  9	 0.0000	 0.0000
 10	 0.0000	 0.0000
 11	 0.0000	 0.0000
 12	 0.0000	 0.0000
 13	 0.0011	 0.0010
 14	 0.0003	 0.0006
 15	 0.0000	 0.0000
 16	 0.0025	 0.0012
 17	 0.0000	 0.0000
 18	 0.0004	 0.0003
 19	 0.0000	 0.0000
 20	 0.0001	 0.0002
 21	 0.0000	 0.0000
 22	 0.0000	 0.0000
 23	 0.0005	 0.0004
 24	 0.0000	 0.0000
 25	 0.0000	 0.0000
 26	 0.0002	 0.0002
 27	 0.0000	 0.0000
 28	 0.0000	 0.0000
 29	 0.0000	 0.0001
 30	 0.0000	 0.0001
 31	 0.0011	 0.0008
 32	 0.0000	 0.0000
 33	 0.0000	 0.0000
 34	 0.0000	 0.0000
 35	 0.0000	 0.0000
 36	 0.0000	 0.0000
 37	 0.0000	 0.0000
 38	 0.0000	 0.0000
 39	 0.0000	 0.0001
 40	 0.0000	 0.0000
 41	 0.0000	 0.0000
 42	 0.0000	 0.0000
 43	 0.0000	 0.0000
 44	 0.0000	 0.0000
 45	 0.0000	 0.0000
 46	 0.0000	 0.0000
 47	 0.0000	 0.0000
 48	 0.00

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


top_k = 2
D = X_full.shape[1]

phi_safe = np.nan_to_num(phi_0_cpi_ad_hoc, nan=-np.inf, posinf=-np.inf, neginf=-np.inf)


top_k = min(top_k, D)
selected_feature_idx = np.argsort(phi_safe)[-top_k:][::-1].tolist()

print(f"Top-{top_k} feature indices by CPI_ad_hoc:")
for rank, j in enumerate(selected_feature_idx, start=1):
    print(f"  #{rank}: idx={j:4d}, CPI_ad_hoc={phi_0_cpi_ad_hoc[j]:.6f}, SE={se_0_cpi_ad_hoc[j]:.6f}")

X_subset = X_full[:, selected_feature_idx]

seed = seed
n_jobs = n_jobs
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")

print("Selected indices:", selected_feature_idx)


Top-2 feature indices by CPI_ad_hoc:
  #1: idx=  16, CPI_ad_hoc=0.002510, SE=0.001235
  #2: idx=  31, CPI_ad_hoc=0.001130, SE=0.000840
Fold 1 accuracy (selected features): 0.9082
Fold 2 accuracy (selected features): 0.9177
Mean accuracy (2-fold, selected features): 0.9130 ± 0.0067
Selected indices: [16, 31]


| Top-k | Accuracy | Std |
|:------|:---------:|:---------:|
| Top-10 | 0.9272 | ±0.0090 |
| Top-8 | 0.9209 | ±0.0000 |
| Top-6 | 0.9161 | ±0.0022 |
| Top-4 | 0.9177 | ±0.0134 |
| Top-2 | 0.9130 | ±0.0067 |

# FDFI

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

top_k = 10

D = X_full.shape[1]

phi_safe = np.nan_to_num(phi_x_cpi, nan=-np.inf, posinf=-np.inf, neginf=-np.inf)

top_k = min(top_k, D)
selected_feature_idx = np.argsort(phi_safe)[-top_k:][::-1].tolist()

print(f"Top-{top_k} feature indices by FDFI:")
for rank, j in enumerate(selected_feature_idx, start=1):
    print(f"  #{rank}: idx={j:4d}, FDFI={phi_x_cpi[j]:.6f}, SE={se_x_cpi[j]:.6f}")

X_subset = X_full[:, selected_feature_idx]

n_jobs = n_jobs
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")

print("Selected indices:", selected_feature_idx)


Top-10 feature indices by FDFI:
  #1: idx= 179, FDFI=0.002618, SE=0.000667
  #2: idx= 320, FDFI=0.002475, SE=0.000876
  #3: idx= 173, FDFI=0.001913, SE=0.000539
  #4: idx= 290, FDFI=0.001684, SE=0.000413
  #5: idx= 795, FDFI=0.001638, SE=0.000521
  #6: idx= 302, FDFI=0.001548, SE=0.000472
  #7: idx=  95, FDFI=0.001484, SE=0.000482
  #8: idx=  21, FDFI=0.001451, SE=0.000578
  #9: idx=1089, FDFI=0.001438, SE=0.000288
  #10: idx= 346, FDFI=0.001336, SE=0.000560
Fold 1 accuracy (selected features): 0.9399
Fold 2 accuracy (selected features): 0.9494
Mean accuracy (2-fold, selected features): 0.9446 ± 0.0067
Selected indices: [179, 320, 173, 290, 795, 302, 95, 21, 1089, 346]


| Top-k | Accuracy | Std |
|:------|:---------:|:---------:|
| Top-10 | 0.9446 | ±0.0067 |
| Top-8 | 0.9415 | ±0.0022 |
| Top-6 | 0.9415 | ±0.0067 |
| Top-4 | 0.9383 | ±0.0022 |
| Top-2 | 0.9399| ±0.0090 |

# DFI

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

top_k = 2
D = X_full.shape[1]

phi_safe = np.nan_to_num(phi_x_dfi, nan=-np.inf, posinf=-np.inf, neginf=-np.inf)

top_k = min(top_k, D)
selected_feature_idx = np.argsort(phi_safe)[-top_k:][::-1].tolist()

print(f"Top-{top_k} feature indices by DFI:")
for rank, j in enumerate(selected_feature_idx, start=1):
    print(f"  #{rank}: idx={j:4d}, DFI={phi_x_dfi[j]:.6f}, SE={se_x_dfi[j]:.6f}")

X_subset = X_full[:, selected_feature_idx]

n_jobs = n_jobs

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")

print("Selected indices:", selected_feature_idx)


Top-2 feature indices by DFI:
  #1: idx=   0, DFI=0.019841, SE=0.000597
  #2: idx= 140, DFI=0.018975, SE=0.000572
Fold 1 accuracy (selected features): 0.9082
Fold 2 accuracy (selected features): 0.9051
Mean accuracy (2-fold, selected features): 0.9066 ± 0.0022
Selected indices: [0, 140]


| Top-k | Accuracy | Std |
|:------|:---------:|:---------:|
| Top-10 | 0.9335 | ±0.0134 |
| Top-8 | 0.9225  | ±0.0022|
| Top-6 | 0.9114 | ±0.0090 |
| Top-4 | 0.9177 | ±0.0000 |
| Top-2 | 0.9066 | ±0.0022 |